# autograd

Variable构造函数需要传入Tensor，同时有两个可选参数

requires_grad(bool)：是否需要求导

variabel支持绝大数tensor操作，但不支持inplace操作

variable.backward(grad_variables=None,retain_graph=None,create_graph=None)

grad_variables ：形状与variable一致，对于y.backward()，grad_variable可以是tensor或序列

retain_graph ：反向传播的时候需要缓存一些中间结果，反向传播之后，这些缓存就被清空，可通过指定这个参数不清空缓存，用来多次反向传播

create_graph ：对反向传播过程再次构建计算图，可通过backward of backward实现求高阶导数

In [1]:
import torch
from torch.autograd import variable

In [2]:
a = torch.ones((3,4), requires_grad = True)
a

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)

In [3]:
b = torch.zeros(3,4)
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [4]:
c = a+b
c

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)

In [5]:
d = c.sum()
d.backward()

In [6]:
a.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [7]:
b.grad

In [8]:
a.requires_grad, b.requires_grad, c.requires_grad #c依赖于a，因此需要求导

(True, False, True)

In [9]:
a.is_leaf, b.is_leaf, c.is_leaf

(True, True, False)

In [10]:
c.grad is None #反向传播结束后中间结果被释放

True

# 计算图

In [11]:
x = torch.ones(1)

In [12]:
w = torch.ones(1, requires_grad = True)

In [13]:
b = torch.ones(1, requires_grad = True)

In [14]:
y = w*x

In [15]:
z = y+b

In [16]:
x.requires_grad, b.requires_grad, w.requires_grad

(False, True, True)

In [17]:
y.requires_grad

True

In [18]:
x.is_leaf, b.is_leaf, w.is_leaf

(True, True, True)

In [19]:
z.is_leaf, y.is_leaf

(False, False)

In [20]:
z.grad_fn #add的反向传播函数为addback

In [21]:
y.grad_fn

In [22]:
#第一个为y，其对应的反向传播函数与y.grad_fn相同,第二个为b，grad_fn为None，但有grad_fn.next_functions,累加梯度
z.grad_fn.next_functions

((<MulBackward0 at 0x7f2b2634ec88>, 0),
 (<AccumulateGrad at 0x7f2b2634f048>, 0))

In [23]:
y.grad_fn.next_functions #w, x

((<AccumulateGrad at 0x7f2b26349518>, 0), (None, 0))

In [24]:
w.grad_fn, b.grad_fn

(None, None)

In [25]:
z.backward(retain_graph=True)

## 查看非叶子节点的grad

In [26]:
x = torch.ones((3), requires_grad = True)
w = torch.randn((3), requires_grad = True)
y = w*x
z = sum(y)

In [27]:
z.backward()

In [28]:
x.grad, w.grad, y.grad #非叶子节点计算完自动清空

(tensor([0.2658, 1.0191, 0.0557]), tensor([1., 1., 1.]), None)